[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/lifecycle-allocation/blob/main/examples/notebooks/tutorial.ipynb)

# Lifecycle Allocation Tutorial

**What stock/bond split should you actually have?**

Most allocation rules are single-variable heuristics -- 60/40, "100 minus your age", target-date funds. This library implements a *lifecycle* approach inspired by [Choi et al. (2024)](https://www.nber.org/papers/w34166): your future earning power (human capital) acts like a bond, and accounting for it changes how much stock risk you should take.

This tutorial walks through the full library in ~10 minutes.

> **Disclaimer:** This is for education and research. It is not investment advice.

In [ ]:
# Install (run this cell on Colab, skip locally if already installed)
import sys

if 'google.colab' in sys.modules:
    !pip install -q git+https://github.com/engineerinvestor/lifecycle-allocation.git

In [ ]:
import matplotlib.pyplot as plt

from lifecycle_allocation import (
    BenefitModelSpec,
    ConstraintsSpec,
    DiscountCurveSpec,
    IncomeModelSpec,
    InvestorProfile,
    MarketAssumptions,
    alpha_star,
    compare_strategies,
    human_capital_pv,
    recommended_stock_share,
    risk_tolerance_to_gamma,
)
from lifecycle_allocation.viz.charts import plot_balance_sheet, plot_strategy_bars

---
## 1. Your First Allocation in 4 Lines

All you need is your age, retirement age, investable wealth, income, and how much risk you're comfortable with (1 = very conservative, 10 = very aggressive).

In [ ]:
profile = InvestorProfile(
    age=30,
    retirement_age=67,
    investable_wealth=100_000,
    after_tax_income=70_000,
    risk_tolerance=5,  # 1-10 scale
)
market = MarketAssumptions(mu=0.05, r=0.02, sigma=0.18)

result = recommended_stock_share(profile, market)

print(f"Recommended stock allocation: {result.alpha_recommended:.1%}")

That's it. But *why* that number? Let's dig in.

---
## 2. Understanding the Result

The `AllocationResult` contains everything: the baseline risky share, unconstrained allocation, human capital, and a plain-English explanation.

In [ ]:
print(f"Baseline risky share (alpha*):  {result.alpha_star:.1%}")
print(f"Human capital (H):              ${result.human_capital:,.0f}")
print(f"Financial wealth (W):           ${profile.investable_wealth:,.0f}")
print(f"H/W ratio:                      {result.human_capital / profile.investable_wealth:.1f}x")
print(f"Unconstrained allocation:       {result.alpha_unconstrained:.1%}")
print(f"Final (clamped to [0, 1]):      {result.alpha_recommended:.1%}")

**Key insight:** A 30-year-old with $100k saved and $70k income has *far* more total wealth than $100k -- their future earnings are worth over $1M in present value. Since human capital behaves like a bond (stable, predictable cash flows), the model says: "You already have a huge bond-like asset. Your investable portfolio should lean heavily into stocks."

In [ ]:
print(result.explain)

---
## 3. The Personal Balance Sheet

The balance sheet chart makes the H/W ratio visual and intuitive.

In [ ]:
fig = plot_balance_sheet(result, profile)
plt.show()

---
## 4. How Does This Compare to Rules of Thumb?

`compare_strategies` benchmarks the lifecycle allocation against common heuristics.

In [ ]:
df = compare_strategies(profile, market)
print(df.to_string(index=False))

In [ ]:
fig = plot_strategy_bars(df)
plt.show()

For a 30-year-old, the lifecycle model suggests *more* equity than all three heuristics -- because it recognizes the massive human capital that rules of thumb ignore.

---
## 5. Three Archetypes: Young, Mid-Career, Near Retirement

Let's see how the recommendation changes across a career.

In [ ]:
archetypes = {
    "Young Saver (25)": InvestorProfile(
        age=25, retirement_age=67, investable_wealth=25_000,
        after_tax_income=70_000, risk_tolerance=7,
        income_model=IncomeModelSpec(type="growth", g=0.02),
    ),
    "Mid-Career (45)": InvestorProfile(
        age=45, retirement_age=67, investable_wealth=500_000,
        after_tax_income=120_000, risk_tolerance=5,
    ),
    "Near Retirement (60)": InvestorProfile(
        age=60, retirement_age=67, investable_wealth=1_000_000,
        after_tax_income=150_000, risk_tolerance=3,
    ),
}

print(f"{'Archetype':<25} {'H/W':>6} {'alpha*':>8} {'Recommended':>12}")
print("-" * 55)
for name, p in archetypes.items():
    r = recommended_stock_share(p, market)
    hw = r.human_capital / p.investable_wealth
    print(f"{name:<25} {hw:>5.1f}x {r.alpha_star:>7.1%} {r.alpha_recommended:>11.1%}")

The pattern is clear:
- **Young savers** have enormous H/W ratios -- the model recommends maximum equity.
- **Mid-career** investors still have significant human capital -- equity stays high.
- **Near retirement** investors have low H/W -- the allocation drops substantially.

This is *qualitatively* similar to target-date funds, but driven by actual numbers rather than a marketing glide path.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for ax, (name, p) in zip(axes, archetypes.items()):
    r = recommended_stock_share(p, market)
    plot_balance_sheet(r, p, ax=ax)
    ax.set_title(name, fontsize=12, fontweight="bold")

plt.tight_layout()
plt.show()

---
## 6. Risk Tolerance: What Does It Feel Like?

The risk tolerance scale (1-10) maps to a risk aversion coefficient (gamma) via a log-linear function. Here's how the allocation changes across the full range.

In [ ]:
print(f"{'RT':>4} {'Gamma':>8} {'alpha*':>8} {'Recommended':>12}")
print("-" * 35)
for rt in range(1, 11):
    gamma = risk_tolerance_to_gamma(rt)
    p = InvestorProfile(
        age=35, retirement_age=67, investable_wealth=200_000,
        after_tax_income=90_000, risk_tolerance=rt,
    )
    r = recommended_stock_share(p, market)
    print(f"{rt:>4} {gamma:>8.2f} {r.alpha_star:>7.1%} {r.alpha_recommended:>11.1%}")

Even the most conservative investor (rt=1, gamma=10) gets a meaningful equity allocation because of human capital. The most aggressive (rt=10, gamma=2) maxes out the 100% cap.

---
## 7. Income Growth Matters

Higher expected income growth means larger human capital, which pushes the allocation higher. Let's compare flat income vs. 2% real growth vs. the CGM education-based profile.

In [ ]:
income_models = {
    "Flat (0% real growth)": IncomeModelSpec(type="flat"),
    "Growth (2% real)": IncomeModelSpec(type="growth", g=0.02),
    "CGM College Profile": IncomeModelSpec(type="profile", education="college"),
}

base = dict(
    age=30, retirement_age=67, investable_wealth=100_000,
    after_tax_income=70_000, risk_tolerance=5,
)

print(f"{'Income Model':<28} {'H':>12} {'H/W':>6} {'Allocation':>12}")
print("-" * 62)
for name, spec in income_models.items():
    p = InvestorProfile(**base, income_model=spec)
    r = recommended_stock_share(p, market)
    hw = r.human_capital / p.investable_wealth
    print(f"{name:<28} ${r.human_capital:>10,.0f} {hw:>5.1f}x {r.alpha_recommended:>11.1%}")

---
## 8. Retirement Benefits Shift the Picture

Social Security or a pension adds bond-like wealth *after* retirement, which can increase the allocation for older investors.

In [ ]:
# Near-retirement investor, without and with Social Security proxy
for benefit_label, benefit_spec in [
    ("No benefits", BenefitModelSpec(type="none")),
    ("$30k/yr benefit", BenefitModelSpec(type="flat", annual_benefit=30_000)),
    ("40% replacement", BenefitModelSpec(type="flat", replacement_rate=0.40)),
]:
    p = InvestorProfile(
        age=60, retirement_age=67, investable_wealth=800_000,
        after_tax_income=100_000, risk_tolerance=4,
        benefit_model=benefit_spec,
    )
    r = recommended_stock_share(p, market)
    alloc = r.alpha_recommended
    print(f"{benefit_label:<20}  H=${r.human_capital:>10,.0f}  Allocation={alloc:.1%}")

Benefits add to human capital, which increases the equity allocation -- even for someone near retirement.

---
## 9. Leverage: When 100% Equity Isn't Enough

The model sometimes recommends *more than 100% equity* -- especially for young investors with huge H/W ratios. By default, the result is clamped to 100%. But you can enable leverage with a borrowing spread to see the unconstrained recommendation.

**This is for educational purposes.** Real-world leverage carries risks not captured by the model.

In [ ]:
young = InvestorProfile(
    age=25, retirement_age=67, investable_wealth=20_000,
    after_tax_income=65_000, risk_tolerance=8,
    income_model=IncomeModelSpec(type="growth", g=0.02),
)

# Without leverage (default)
r_no_lev = recommended_stock_share(young, market)
unconstrained = r_no_lev.alpha_unconstrained
print(f"Without leverage:  {r_no_lev.alpha_recommended:.1%}  (unconstrained: {unconstrained:.1%})")

# With leverage, 150 bps borrowing spread
market_lev = MarketAssumptions(mu=0.05, r=0.02, sigma=0.18, borrowing_spread=0.015)
constraints_lev = ConstraintsSpec(allow_leverage=True, max_leverage=2.0)
r_lev = recommended_stock_share(young, market_lev, constraints=constraints_lev)
print(f"With leverage:     {r_lev.alpha_recommended:.1%}  (max_leverage=2.0, spread=150bps)")
print(f"Leverage applied:  {r_lev.leverage_applied}")

In [ ]:
# The explanation includes leverage risk disclosures when leverage is active
if r_lev.leverage_applied:
    print(r_lev.explain)

---
## 10. Sensitivity: How Assumptions Change the Answer

Let's see how the allocation moves when we change key inputs one at a time.

In [ ]:
base_profile = InvestorProfile(
    age=35, retirement_age=67, investable_wealth=200_000,
    after_tax_income=90_000, risk_tolerance=5,
)
base_market = MarketAssumptions(mu=0.05, r=0.02, sigma=0.18)
base_result = recommended_stock_share(base_profile, base_market)

print(f"Base case: {base_result.alpha_recommended:.1%}\n")

# Vary expected return
print("Varying expected stock return (mu):")
for mu in [0.03, 0.04, 0.05, 0.06, 0.07, 0.08]:
    m = MarketAssumptions(mu=mu, r=0.02, sigma=0.18)
    r = recommended_stock_share(base_profile, m)
    bar = "#" * int(r.alpha_recommended * 40)
    print(f"  mu={mu:.0%}  {r.alpha_recommended:>5.1%}  {bar}")

print("\nVarying stock volatility (sigma):")
for sigma in [0.12, 0.15, 0.18, 0.22, 0.25, 0.30]:
    m = MarketAssumptions(mu=0.05, r=0.02, sigma=sigma)
    r = recommended_stock_share(base_profile, m)
    bar = "#" * int(r.alpha_recommended * 40)
    print(f"  sigma={sigma:.0%}  {r.alpha_recommended:>5.1%}  {bar}")

print("\nVarying risk tolerance:")
for rt in [1, 3, 5, 7, 10]:
    p = InvestorProfile(
        age=35, retirement_age=67, investable_wealth=200_000,
        after_tax_income=90_000, risk_tolerance=rt,
    )
    r = recommended_stock_share(p, base_market)
    bar = "#" * int(r.alpha_recommended * 40)
    print(f"  rt={rt:<2}    {r.alpha_recommended:>5.1%}  {bar}")

---
## 11. Discount Rate Sensitivity

The discount rate for human capital is a key assumption. A higher rate shrinks H, which reduces the equity allocation.

In [ ]:
print(f"{'Discount Rate':>14} {'H':>12} {'H/W':>6} {'Allocation':>12}")
print("-" * 48)
for rate in [0.00, 0.01, 0.02, 0.03, 0.04, 0.05]:
    curve = DiscountCurveSpec(type="constant", rate=rate)
    r = recommended_stock_share(base_profile, base_market, curve=curve)
    hw = r.human_capital / base_profile.investable_wealth
    print(f"{rate:>13.0%} ${r.human_capital:>10,.0f} {hw:>5.1f}x {r.alpha_recommended:>11.1%}")

---
## 12. Working Directly with the Building Blocks

The library exposes the individual functions so you can compose your own analysis.

In [ ]:
# Compute just the baseline risky share
a, leverage_used, drag = alpha_star(market, gamma=4.0)
print(f"alpha* for gamma=4: {a:.1%}")

# Compute just human capital
h = human_capital_pv(base_profile)
print(f"Human capital PV:   ${h:,.0f}")

# The log-return variant gives a slightly different alpha*
a_log, _, _ = alpha_star(market, gamma=4.0, variant="log_return")
print(f"alpha* (log-return): {a_log:.1%}  (vs Merton: {a:.1%})")

---
## 13. Allocation Over a Lifetime

Let's simulate how the allocation evolves as someone ages, assuming their wealth grows at 5% per year and income stays flat.

In [ ]:
from lifecycle_allocation.core.strategies import (
    strategy_n_minus_age,
    strategy_parametric_tdf,
    strategy_sixty_forty,
)

ages = list(range(25, 80))
choi_allocs = []
wealth = 25_000.0

for age in ages:
    p = InvestorProfile(
        age=age, retirement_age=67,
        investable_wealth=max(wealth, 1_000),  # floor to avoid W<=0
        after_tax_income=70_000 if age < 67 else None,
        risk_tolerance=5,
    )
    r = recommended_stock_share(p, market)
    choi_allocs.append(r.alpha_recommended)
    wealth = wealth * 1.05 + (70_000 * 0.15 if age < 67 else -40_000)  # rough accumulation

nma_allocs = [strategy_n_minus_age(a) for a in ages]
tdf_allocs = [strategy_parametric_tdf(a) for a in ages]
sf_allocs = [strategy_sixty_forty(a) for a in ages]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(ages, choi_allocs, linewidth=2.5, label="Lifecycle (Choi)", color="#2196F3")
ax.plot(ages, sf_allocs, "--", label="60/40", color="#FF9800")
ax.plot(ages, nma_allocs, "--", label="100-minus-age", color="#9C27B0")
ax.plot(ages, tdf_allocs, "--", label="Target-Date Fund", color="#F44336")
ax.set_xlabel("Age", fontsize=12)
ax.set_ylabel("Stock Allocation", fontsize=12)
ax.set_title("Equity Allocation Over a Lifetime", fontsize=14, fontweight="bold")
ax.legend(fontsize=10)
ax.set_ylim(0, 1.05)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0%}"))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.axhline(y=1.0, color="gray", linewidth=0.5, linestyle=":")
plt.tight_layout()
plt.show()

The lifecycle model starts high (because of human capital) and declines as financial wealth grows and human capital shrinks. The shape is similar to a target-date fund, but the *level* is driven by individual circumstances rather than a one-size-fits-all glide path.

---
## 14. Using YAML Profiles and the CLI

For reproducible analysis, you can define profiles in YAML and use the command-line interface.

Here's what a YAML profile looks like:

```yaml
age: 25
retirement_age: 67
investable_wealth: 25000
after_tax_income: 70000
risk_tolerance: 7

income_model:
  type: growth
  g: 0.02

benefit_model:
  type: none

market:
  mu: 0.05
  r: 0.02
  sigma: 0.18
  real: true
```

Then from the terminal:

```bash
lifecycle-allocation alloc \
    --profile examples/profiles/young_saver.yaml \
    --out ./output \
    --report
```

This generates `allocation.json`, `summary.md`, and charts in `output/charts/`.

---
## Summary

| Concept | What it means |
|---|---|
| **Human capital (H)** | PV of your future earnings -- acts like a bond |
| **H/W ratio** | How much of your total wealth is human capital |
| **alpha\*** | Baseline equity allocation from risk preferences and market assumptions |
| **Recommended allocation** | alpha\* adjusted for H/W, clamped to constraints |

**The core insight:** Young workers should hold more equity than rules of thumb suggest -- not because they can "afford to lose it," but because most of their wealth is already in a bond-like asset (their career).

As always: this is for education, not advice. Your actual allocation should consider your full financial picture, tax situation, risk capacity, and goals.